**1. Look up the Adam optimization functions in PyTorch https://pytorch.org/docs/stable/optim.html . How does it work? Try at least one other optimization function with the diabetes dataset shown in class. How does the model perform with the new optimizer? Did it perform better or worse than Adam? Why do you think that is?**

- Optimization functions using the adam algorithm
    - They are a good replacement for stochastic gradient descent (in other words, learning rate changes for Adam)
    - They utilize the best qualities of other algorithms (Adaptive Gradient Algorithm - AdaGrad and Root Mean Square Propagation - RMSProp) for problems with a lot of "noise" and sparse gradients
    - It's a versatile algorithm that adjusts the learning rates to ensure reasonable values through weight optimization
    - It usually acheives good results for deep learning models
    
Our group tested a different optimizer called Adamax, which is a variant of Adam. It performed better, overall. The accuracy score between the two classification reports was only 1 point different. However, the Adamax optimizer increased the recall value from 48% to 62%. This is quite a large difference, especially considering it is the only change we made in the model! I think it performs better because, according to my quick research, Adamax is updating parameters less often, so there is less opportunity for "noise."

**Code from class**

We changed the optimizer from Adam to Adamax

In [27]:
# Test out code
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F #where the activation functions are

diabetes_df = pd.read_csv('diabetes.csv')

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [28]:
#create tensors = matrices 
X_train = torch.FloatTensor(X_train) 
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [29]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__() #super is a computed indirect reference. So, it isolates changes
        # and makes sure that children in the layers of multiple inheritence are calling
        #the right parents
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [30]:
# manually set the seed
torch.manual_seed(42)

#create instance of model
ann = ANN_Model()

#loss function
loss_function = nn.CrossEntropyLoss()

**Change made here!**

In [31]:
#optimizer
optimizer = torch.optim.Adamax(ann.parameters(),lr=0.01)

In [130]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
#     if epoch % 10 == 1:
#         print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

In [34]:
#predictions
y_pred = []

with torch.no_grad(): #decreases memory consumption
    for i, data in enumerate(X_test):
        prediction = ann(data)
        y_pred.append(prediction.argmax()) #returns index with max element in each prediction set

**Revised classification report**

In [35]:
from sklearn.metrics import classification_report

# print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.75      0.76       150
           1       0.57      0.62      0.59        81

    accuracy                           0.70       231
   macro avg       0.68      0.68      0.68       231
weighted avg       0.71      0.70      0.70       231



**2. Write a function that lists and counts the number of divisors for an input value.**

**Example 1:**

Input: 5

Output: “There are 2 divisors: 1 and 5”

**Example 2:**

Input: 40

Output: “There are 8 divisors: 1, 2, 4, 5, 8, 10, 20, and 40”

In [127]:
# Define divisors function
def divisors():
    
    # error handling
    try:
    
        # Create input to enter a number
        number = int(input('Enter an integer '))

        # Create empty list to store divisors
        div_list = []

        # iterate input to find the divisors for that number
        for d in range(1, number+1):
            # Check to see if there is no remainder
            if number%d == 0:
                # append the divisor to the list
                div_list.append(d)

        # count the number of divisors
        count = len(div_list)
        # convert the div_list to a string
        string_divs = list(map(str, div_list))
        # reformat to add commas and "and" between last values
        organized_divs = ' and '.join(', '.join(string_divs).rsplit(', ', 1))
        
        # print input so user remembers their value
        print("Input: " + str(number))
        
        # Output for user
        return f'There are {count} divisors: {organized_divs}'
    
    # print error if wrong input is given
    except Exception as e:
        print('Try again with an integer')

In [122]:
# Call function with example 1: 
# Input = 5
divisors()

Enter an integer 5
Input: 5


'There are 2 divisors: 1 and 5'

In [119]:
# Call function with example 2: 
# Input 40
divisors()

Enter an integer 40
Input: 40


'There are 8 divisors: 1, 2, 4, 5, 8, 10, 20 and 40'

In [128]:
# Error
# Input 5.5
divisors()

Enter an integer 5.5
Try again with an integer


In [129]:
# Error
# Input watermelon
divisors()

Enter an integer watermelon
Try again with an integer
